In [1]:
%pip install opencv-python numpy openmim torchvision
from pprint import pprint
import numpy as np
import cv2
import csv
import json
import os
import sys
import shutil
import pickle

# Classes are defined in process_dataset/common.py
from process_dataset.common import classes, classes_dict, datasets_path, dataset_pickle_filepath

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Additional paths

In [2]:
import paths

with open("./paths.py") as f:
    print(f.read())

# proj_path = os.getcwd()

# drive_dirpath = os.path.join(proj_path, "drive")
# drive_traffic_cams_path = os.path.join(drive_dirpath, "traffic_cams")

# mmdetection_path = os.path.join(proj_path, "..", "mmdetection")

# working_dirpath = os.path.join(proj_path, "working_dir")

# process_dataset_dirpath = os.path.join(proj_path, "process_dataset")

# !mkdir -p $working_dirpath

# ! Do this in a separate terminal:
# !mkdir -p $drive_dirpath
# !rclone mount remote: $drive_dirpath

# Don't forget to unmount when you're finished

# TODO assert everything is in the right place

import os

proj_path = os.getcwd()

drive_dirpath = os.path.join(proj_path, "drive")
drive_traffic_cams_path = os.path.join(drive_dirpath, "traffic_cams")

mmdetection_path = os.path.join(proj_path, "..", "mmdetection")

working_dirpath = os.path.join(proj_path, "working_dir2")

process_dataset_dirpath = os.path.join(proj_path, "process_dataset")


mmdetection_config_path = os.path.join(mmdetection_path, "configs")
mmdetection_checkpoint_path = os.path.join(mmdetection_path, "checkpoints")

# model_config_filename = "yolox_x_8x8_300e_coco.py"
# model_config_filename = "yolox_s_8x8_300e_coco.py"
model_config_filename = "yolox_s_8x8_300e_coco_custom.py"
model_config_filepath = os.path.join(mmdetection_config_path, "yolox", model_config_filename)

# And YOLOX-x checkpoint
# model_checkpoint_filename = "yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth"
model_checkpoint_filename = "yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth"
model_checkpoint_filepath = os.path.join(mmdetection_pat

# Installing mmdetection

In [3]:
!mim install mmcv-full
%cd $paths.mmdetection_path/..
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
%pip install -e .
%cd $paths.proj_path

/usr/local/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.openmmlab.com/mmcv/dist/cu117/torch1.13.0/index.html
ERROR: Error while checking for conflicts. Please file an issue on pip's issue tracker: https://github.com/pypa/pip/issues/new
Traceback (most recent call last):
  File "/home/tedro/.local/lib/python3.11/site-packages/pip/_internal/commands/install.py", line 618, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tedro/.local/lib/python3.11/site-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tedro/.local/lib/pyth

In [4]:
#@title Verify installation
import mmdet
print(mmdet.__version__)

/home/tedro/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.25.3


/home/tedro/.local/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [5]:
from mmcv import collect_env
collect_env()

{'sys.platform': 'linux',
 'Python': '3.9.2 (default, Feb 28 2021, 17:03:44) [GCC 10.2.1 20210110]',
 'CUDA available': True,
 'GPU 0': 'NVIDIA GeForce GTX 1080 Ti',
 'CUDA_HOME': '/usr/local/cuda',
 'NVCC': 'Cuda compilation tools, release 11.8, V11.8.89',
 'GCC': 'x86_64-linux-gnu-gcc (Debian 10.2.1-6) 10.2.1 20210110',
 'PyTorch': '1.13.0+cu117',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 9.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.7\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_

# Download YOLOX-s config and checkpoint (pre-trained)

In [6]:
# Memory requirements for YOLOX models: 
# YOLOX-s	7.6 GB
# YOLOX-l	19.9 GB
# YOLOX-x	28.1 GB

# url = "https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/" + model_checkpoint_filename
url = "https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_s_8x8_300e_coco/" + paths.model_checkpoint_filename
!wget -c $url -O $paths.model_checkpoint_filepath

--2022-12-02 14:51:56--  https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_s_8x8_300e_coco/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.254.186.218
Connecting to download.openmmlab.com (download.openmmlab.com)|47.254.186.218|:443... connected.
HTTP request sent, awaiting response... 200 OK

    The file is already fully retrieved; nothing to do.



In [7]:
# https://colab.research.google.com/github/ZwwWayne/mmdetection/blob/update-colab/demo/MMDet_Tutorial.ipynb#scrollTo=hamZrlnH-YDD
from mmdet.apis import set_random_seed
from mmcv import Config

cfg = Config.fromfile(paths.model_config_filepath)

# print(f'Initial config:\n{cfg.pretty_text}')

if paths.last_checkpoint_filepath:
    cfg.resume_from = paths.last_checkpoint_filepath
else:
    cfg.load_from = paths.model_checkpoint_filepath

cfg.work_dir = paths.working_dirpath

# cfg.data_root = dataset_mio_tcd_path
cfg.data_root = datasets_path

# cfg.dataset_type = "CustomDataset"

# img_prefix = dataset_mio_tcd_path
img_prefix = datasets_path

# dataset_filepath = os.path.join(dataset_mio_tcd_path, "gt.pickle")
dataset_filepath = os.path.join(datasets_path, "dataset.pickle")
train_filepath = os.path.join(datasets_path, "train.pickle")
val_filepath = os.path.join(datasets_path, "val.pickle")
test_filepath = os.path.join(datasets_path, "test.pickle")

# cfg.train_dataset.dataset.type = "CustomDataset"
# cfg.train_dataset.dataset.ann_file = train_filepath
# cfg.train_dataset.dataset.img_prefix = img_prefix
cfg.data.train.dataset.type = "CustomDataset"
cfg.data.train.dataset.ann_file = train_filepath
cfg.data.train.dataset.img_prefix = img_prefix
cfg.data.val.type = "CustomDataset"
cfg.data.val.ann_file = val_filepath
cfg.data.val.img_prefix = img_prefix
cfg.data.test.type = "CustomDataset"
cfg.data.test.ann_file = test_filepath
cfg.data.test.img_prefix = img_prefix

# TODO try class balanced
cfg.data.train.dataset = {
        "type": 'ClassBalancedDataset',
        # "oversample_thr": 1e-3,
        "oversample_thr": 0.1,
        "dataset": cfg.data.train.dataset
    }

cfg.train_pipeline = [
    dict(type='Mosaic', 
         img_scale=cfg.img_scale, 
         pad_val=114.0),
    dict(
        type='RandomAffine',
        max_rotate_degree=15,
        scaling_ratio_range=(0.3, 2),
        max_shear_degree=10,
        # min_bbox_size=8, # No need. Done in FilterAnnotations
        border=(-cfg.img_scale[0] // 2, -cfg.img_scale[1] // 2)),
    dict(
        type='MixUp',
        img_scale=cfg.img_scale,
        ratio_range=(0.8, 1.6),
        pad_val=114.0),
    dict(type='YOLOXHSVRandomAug'),
    dict(type='RandomFlip', 
         flip_ratio=0.5,
         direction="horizontal"), # (horizontal is implicit)
    # According to the official implementation, multi-scale
    # training is not considered here but in the
    # 'mmdet/models/detectors/yolox.py'.
    dict(type='Resize', 
         img_scale=cfg.img_scale, 
         keep_ratio=True),
    dict(
        type='Pad',
        pad_to_square=True,
        # If the image is three-channel, the pad value needs
        # to be set separately for each channel.
        pad_val=dict(img=(114.0, 114.0, 114.0))),
    dict(type='FilterAnnotations', 
         min_gt_bbox_wh=(8, 8), 
         keep_empty=False),
    dict(type='DefaultFormatBundle'),
    dict(type="PhotoMetricDistortion"),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
cfg.train_dataset.pipeline = cfg.train_pipeline

cfg.gpu_ids = [0]
cfg.device = "cuda"

cfg.seed = 0
set_random_seed(0, deterministic=False)

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
# cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None

cfg.log_config.interval = 50

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'

# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1

cfg.max_epochs = 15
cfg.runner = dict(type='EpochBasedRunner', max_epochs=cfg.max_epochs)

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]

# TODO?
# cfg.workflow = [('train', 1), ('val', 1)]

cfg.pop("train_dataset")
cfg.pop("train_pipeline")
cfg.pop("test_pipeline")

# Tried this but it didn't help with the mmdetection's concat problem
# cfg.data.train.dataset["filter_empty_gt"] = True

[{'type': 'LoadImageFromFile'},
 {'type': 'MultiScaleFlipAug',
  'img_scale': (640, 640),
  'flip': False,
  'transforms': [{'type': 'Resize', 'keep_ratio': True},
   {'type': 'RandomFlip'},
   {'type': 'Pad',
    'pad_to_square': True,
    'pad_val': {'img': (114.0, 114.0, 114.0)}},
   {'type': 'DefaultFormatBundle'},
   {'type': 'Collect', 'keys': ['img']}]}]

In [8]:
print(f'Config:\n{cfg.pretty_text}')

Config:
optimizer = dict(
    type='SGD',
    lr=0.01,
    momentum=0.9,
    weight_decay=0.0005,
    nesterov=True,
    paramwise_cfg=dict(norm_decay_mult=0.0, bias_decay_mult=0.0))
optimizer_config = dict(grad_clip=None)
lr_config = dict(
    policy='YOLOX',
    warmup=None,
    by_epoch=False,
    warmup_by_epoch=True,
    warmup_ratio=1,
    warmup_iters=5,
    num_last_epochs=15,
    min_lr_ratio=0.05)
runner = dict(type='EpochBasedRunner', max_epochs=15)
checkpoint_config = dict(interval=1)
log_config = dict(
    interval=50,
    hooks=[dict(type='TextLoggerHook'),
           dict(type='TensorboardLoggerHook')])
custom_hooks = [
    dict(type='YOLOXModeSwitchHook', num_last_epochs=15, priority=48),
    dict(type='SyncNormHook', num_last_epochs=15, interval=10, priority=48),
    dict(
        type='ExpMomentumEMAHook',
        resume_from=None,
        momentum=0.0001,
        priority=49)
]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = '/hom

In [9]:
# https://colab.research.google.com/github/ZwwWayne/mmdetection/blob/update-colab/demo/MMDet_Tutorial.ipynb#scrollTo=hamZrlnH-YDD
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# cfg.model.CLASSES = classes

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# TODO does this help with "Class names are not saved in the checkpoint"?
# cfg.model.CLASSES = datasets[0].CLASSES
 
# Build the detector
# model = build_detector(cfg.model, test_cfg=cfg.data.test)
model = build_detector(cfg.model)

# Train
model.CLASSES = datasets[0].CLASSES
train_detector(model, datasets, cfg)

2022-12-02 14:52:07,413 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-12-02 14:52:07,435 - mmdet - INFO - load checkpoint from local path: /home/tedro/Desktop/d_projekty/bp/proj/working_dir2/latest.pth
2022-12-02 14:52:07,779 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_backbone_stem_conv_conv_weight, ema_backbone_stem_conv_bn_weight, ema_backbone_stem_conv_bn_bias, ema_backbone_stem_conv_bn_running_mean, ema_backbone_stem_conv_bn_running_var, ema_backbone_stem_conv_bn_num_batches_tracked, ema_backbone_stage1_0_conv_weight, ema_backbone_stage1_0_bn_weight, ema_backbone_stage1_0_bn_bias, ema_backbone_stage1_0_bn_running_mean, ema_backbone_stage1_0_bn_running_var, ema_backbone_stage1_0_bn_num_batches_tracked, ema_backbone_stage1_1_main_conv_conv_weight, ema_backbone_stage1_1_main_conv_bn_weight, ema_backbone_stage1_1_main_conv_bn_bias, ema_backbone_stage1_1_main_conv_bn_running_mean, 

KeyboardInterrupt: 

In [ ]:
# from mmdet.apis import single_gpu_test
# from mmdet.datasets import build_dataloader, build_dataset
# from mmdet.utils import build_dp

# data_loader = build_dataloader(build_dataset(cfg.data.test), samples_per_gpu=64, workers_per_gpu=1)
# dp = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)
# outputs = single_gpu_test(dp, data_loader, out_dir=paths.working_dirpath)